# SKEL-basic

SKEL unifies the SMPL model and BSM model, which has the ability to represent realistic human skin and realistic human skeleton with the same set of parameters.

In this notebook, we will introduce the basic usage of `skel` model.

## Preparation

You need to follow the instruction from SKEL's official website to install the SKEL model in the third_party folder. You can first check the instruction [here](../third_party/prepare_SKEL.md).

## Tutorials

### Environment Preparation

In [ ]:
# Packages you may use very often.
import torch
import numpy as np
from smplx import SMPL
from skel.skel_model import SKEL
from pytorch3d import transforms  # You may use this package when performing rotation representation transformation.

# Things you don't need to care about. They are just for driving the tutorials.
from lib.utils.path_manager import PathManager
from lib.viewer.wis3d_utils import HWis3D as Wis3D
from lib.skeleton import Skeleton_SMPL24

pm = PathManager()

### Load SMPL model

In [3]:
skel_models = {}
genders = ['male', 'female']

for gender in genders:
    skel_models[gender] = SKEL(
            model_path = pm.inputs / 'body_models' / 'skel_models_v1.1',
            gender     = gender,
        )

In [4]:
# Prepare some parameters for later inference.
B = 10
skel_model : SKEL = skel_models['male']  # use male for example

# Prepare mesh template for later visualization.
# Tips: mesh = vertices + faces, and the faces are the indices of vertices, which won't change across SKEL's outputs.
skin_mesh_temp : np.ndarray = skel_model.skin_f  # (13776, 3)
print(f'skin_mesh_temp.shape: {skin_mesh_temp.shape}')
skel_mesh_temp : np.ndarray = skel_model.skel_f  # (126665, 3)
print(f'skel_mesh_temp.shape: {skel_mesh_temp.shape}')

skin_mesh_temp.shape: torch.Size([13776, 3])
skel_mesh_temp.shape: torch.Size([126665, 3])


As you can see, the SKEL model has 2 types of mesh, one is SMPL style skin mesh, and the other is BSM style skeleton mesh. And the topology of skeleton mesh is far more complex than skin's. Which means, produce the vertices of skeleton is much more expensive (in memory) than simply skin's. We will see that SKEL allows us to turn off the skeleton's vertices.

### SKEL Inference

In [ ]:
# Inference.
skel_out = skel_model(
        betas    = torch.zeros(B, 10),  # shape coefficients
        poses    = torch.zeros(B, 46),  # Euler-angle representation
        trans    = torch.zeros(B, 3),
        skelmesh = True,  # default `True`, turn off this to save memory
    )

# Check output.
joints     : torch.Tensor = skel_out.joints      # (B, 45, 3)
skin_verts : torch.Tensor = skel_out.skin_verts  # (B, 6890, 3)
skel_verts : torch.Tensor = skel_out.skel_verts  # (B, 247252, 3)
print(f'joint.shape: {tuple(joints.shape)}')
print(f'skin_verts.shape: {tuple(skin_verts.shape)}')
print(f'skel_verts.shape: {tuple(skel_verts.shape)}')

joint.shape: (10, 24, 3)
skin_verts.shape: (10, 6890, 3)
skel_verts.shape: (10, 247252, 3)


As you can see, the number of vertices for skeleton is much larger than skin. This will cause negative impact when you want to perform batch inference. Fortunately, we can turn off the skeleton's vertices to save memory.

#### SKEL Skeleton

The joints definition of SKEL is as below. 

As you can see, different from SMPL's, not all joints of SKEL has 3 degrees of freedom. In the figure below, each joints is annotated with: `<joint_id> <joint_name>(<DoF>)`.

![](assets/SKEL-joints.png)

And the bones (edge in kinematic tree) and joints definition are shown below.

In [ ]:
chains = [
    [0, 1, 4, 7, 10],     # left leg
    [0, 2, 5, 8, 11],     # right leg
    [0, 3, 6, 9, 12, 15], # spine & head
    [12, 13, 16, 18, 20], # left arm
    [12, 14, 17, 19, 21], # right arm
]
bones = [
    [ 0,  1], [ 1,  4], [ 4,  7], [ 7, 10],           # left leg
    [ 0,  2], [ 2,  5], [ 5,  8], [ 8, 11],           # right leg
    [ 0,  3], [ 3,  6], [ 6,  9], [ 9, 12], [12, 15], # spine & head
    [12, 13], [13, 16], [16, 18], [18, 20],           # left arm
    [12, 14], [14, 17], [17, 19], [19, 21],           # right arm
]

# 24 joints ~ 46 DoF
components = [
    {'qid': 0, 'name': 'pelvis', 'jid': 0},
    {'qid': 1, 'name': 'pelvis', 'jid': 0},
    {'qid': 2, 'name': 'pelvis', 'jid': 0},
    {'qid': 3, 'name': 'femur-r', 'jid': 1},
    {'qid': 4, 'name': 'femur-r', 'jid': 1},
    {'qid': 5, 'name': 'femur-r', 'jid': 1},
    {'qid': 6, 'name': 'tibia-r', 'jid': 2},
    {'qid': 7, 'name': 'talus-r', 'jid': 3},
    {'qid': 8, 'name': 'calcn-r', 'jid': 4},
    {'qid': 9, 'name': 'toes-r', 'jid': 5},
    {'qid': 10, 'name': 'femur-l', 'jid': 6},
    {'qid': 11, 'name': 'femur-l', 'jid': 6},
    {'qid': 12, 'name': 'femur-l', 'jid': 6},
    {'qid': 13, 'name': 'tibia-l', 'jid': 7},
    {'qid': 14, 'name': 'talus-l', 'jid': 8},
    {'qid': 15, 'name': 'calcn-l', 'jid': 9},
    {'qid': 16, 'name': 'toes-l', 'jid': 10},
    {'qid': 17, 'name': 'lumbar', 'jid': 11},
    {'qid': 18, 'name': 'lumbar', 'jid': 11},
    {'qid': 19, 'name': 'lumbar', 'jid': 11},
    {'qid': 20, 'name': 'thorax', 'jid': 12},
    {'qid': 21, 'name': 'thorax', 'jid': 12},
    {'qid': 22, 'name': 'thorax', 'jid': 12},
    {'qid': 23, 'name': 'head', 'jid': 13},
    {'qid': 24, 'name': 'head', 'jid': 13},
    {'qid': 25, 'name': 'head', 'jid': 13},
    {'qid': 26, 'name': 'scapula-r', 'jid': 14},
    {'qid': 27, 'name': 'scapula-r', 'jid': 14},
    {'qid': 28, 'name': 'scapula-r', 'jid': 14},
    {'qid': 29, 'name': 'humerus-r', 'jid': 15},
    {'qid': 30, 'name': 'humerus-r', 'jid': 15},
    {'qid': 31, 'name': 'humerus-r', 'jid': 15},
    {'qid': 32, 'name': 'ulna-r', 'jid': 16},
    {'qid': 33, 'name': 'radius-r', 'jid': 17},
    {'qid': 34, 'name': 'hand-r', 'jid': 18},
    {'qid': 35, 'name': 'hand-r', 'jid': 18},
    {'qid': 36, 'name': 'scapula-l', 'jid': 19},
    {'qid': 37, 'name': 'scapula-l', 'jid': 19},
    {'qid': 38, 'name': 'scapula-l', 'jid': 19},
    {'qid': 39, 'name': 'humerus-l', 'jid': 20},
    {'qid': 40, 'name': 'humerus-l', 'jid': 20},
    {'qid': 41, 'name': 'humerus-l', 'jid': 20},
    {'qid': 42, 'name': 'ulna-l', 'jid': 21},
    {'qid': 43, 'name': 'radius-l', 'jid': 22},
    {'qid': 44, 'name': 'hand-l', 'jid': 23},
    {'qid': 45, 'name': 'hand-l', 'jid': 23},
]

### SKEL v.s. SMPL

In [6]:
rnd_trans = torch.rand(B, 3)

# Generate the SMPL T-pose.
smpl_model = SMPL(model_path=pm.inputs/'body_models'/'smpl', gender="male")
smpl_out = smpl_model(
        betas         = torch.zeros(B, 10),     # shape coefficients
        global_orient = torch.zeros(B, 1, 3),   # axis-angle representation
        body_pose     = torch.zeros(B, 23, 3),  # axis-angle representation
        transl        = rnd_trans,
    )
smpl_verts = smpl_out.vertices
# smpl_joints = smpl_out.joints

skel_out = skel_model(
        betas    = torch.zeros(B, 10),  # shape coefficients
        poses    = torch.zeros(B, 46),  # Euler-angle representation
        trans    = rnd_trans,
        skelmesh = False,
    )
skel_verts = skel_out.skin_verts
# skel_joints = skel_out.joints

In [7]:
skel_vs_smpl_wis3d = Wis3D(
        pm.outputs / 'wis3d',
        'SMPL v.s. SKEL',
    )

# Visualize SMPL results.
skel_vs_smpl_wis3d.add_motion_verts(
    verts  = smpl_verts,
    name   = f'smpl_T_pose',
    offset = 0,
)

# Visualize SMPL-H results.
skel_vs_smpl_wis3d.add_motion_verts(
    verts  = skel_verts,
    name   = f'skel_T_pose',
    offset = 0,
)


# Related Works

Check [HSMR: Reconstructing Humans with a Biomechanically Accurate Skeleton](https://isshikihugh.github.io/HSMR/) for regressing SKEL parameters from single images.